In [32]:
import xml.etree.cElementTree as ET
import pprint
import re
import json
from pymongo import MongoClient

In [33]:
OSM_FILE = "plaisir.osm"

In [34]:
SAMPLE_FILE = "sample.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

# First view of the file

In [35]:
tree = ET.parse("sample.osm")
root = tree.getroot()

In [36]:
print "\nChildren of root:"
childs = set()
for child in root:
    childs.add(child.tag)
print childs


Children of root:
set(['node', 'relation', 'way'])


In [37]:
print "\nChildren of childrens of root"
children = {}
children["node"] = set()
children["relation"] = set()
children["way"] = set()
for child in root:
    for i in child:
        children[child.tag].add(i.tag)
print children


Children of childrens of root
{'node': set(['tag']), 'relation': set(['member', 'tag']), 'way': set(['tag', 'nd'])}


# Audit of the file

In [72]:
tags = {}
for event, elem in ET.iterparse("sample.osm"):
    if elem.tag not in tags:
        tags[elem.tag]= 1
    else:
        tags[elem.tag] += 1
print tags

{'node': 33069, 'nd': 46051, 'member': 1718, 'tag': 14808, 'relation': 45, 'way': 5618, 'osm': 1}


amenity
addr:street
building
shop

https://wiki.openstreetmap.org/wiki/Map_Features#Shop